In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
trades = pd.read_csv("trades.csv", low_memory=False).drop("Unnamed: 0", axis = 1)
trades.Quote = trades.Quote.str.replace(",", "").astype(float)
trades["Currency"] = trades["Currency"].replace("WETH", "ETH")
trades = trades[(trades["Currency"]=="ETH") & (trades["Quote"]>0)]

In [3]:
multi_trades = trades[trades[["NFT", "Token ID"]].duplicated()].sort_values(["NFT", "Token ID"]).reset_index(drop=True).rename(columns={"Date Time (UTC)":"timestamp", "Quote":"price_eth"})

In [4]:
multi_trades["nft_id"] = multi_trades[["NFT", "Token ID"]].agg('_'.join, axis=1) 

In [5]:
multi_trades_long = multi_trades.groupby("nft_id").head(2).set_index(["nft_id"])[["timestamp", "price_eth"]]

In [6]:
nft_count = len(multi_trades_long.reset_index()["nft_id"].unique())
"NFTs:", nft_count

('NFTs:', 184893)

In [7]:
trade_count = len(multi_trades_long["price_eth"].tolist())
"Trades:", trade_count

('Trades:', 263914)

In [8]:
multi_trades_long.describe()

,price_eth
count,2.639140e+05
mean,2.516868e+00
std,3.553862e+02
min,1.000000e-18
25%,1.750000e-01
50%,3.999000e-01
75%,9.700000e-01
max,1.770000e+05


In [33]:
mtl = multi_trades_long.reset_index().sort_values("nft_id")

In [34]:
mtl["first_trade"]=None
mtl["second_trade"]=None
#multi_trades_long["profit"]=""

In [39]:
i = 0
for nft in list(mtl["nft_id"].unique()):
    mtl.loc[i, "first_trade"] = mtl.loc[i, "price_eth"]
    mtl.loc[i+1, "second_trade"] = mtl.loc[i+1, "price_eth"]
    i+=2

KeyError: 263914

In [16]:
mtl.loc[0, "first_trade"] = mtl[mtl["nft_id"]=="#LetsWalk_1"].loc[0, "price_eth"]

In [41]:
mtl["second_trade"] = mtl["second_trade"].shift(-1)

In [43]:
mtl["first_timestamp"] = mtl["timestamp"]
mtl["second_timestamp"] = mtl["timestamp"].shift(-1)

In [53]:
mtl = mtl[(mtl["second_trade"].notna()) & (mtl["first_trade"].notna())].drop(["timestamp", "price_eth"], axis=1).set_index("nft_id")

In [55]:
mtl["profit"] = mtl["second_trade"] / mtl["first_trade"] -1

In [65]:
mtl.set_index("nft_id").to_csv("profit_per_nft.csv")

In [66]:
nft_profits = pd.read_csv("profit_per_nft.csv")

In [69]:
nft_profits

,nft_id,first_trade,second_trade,first_timestamp,second_timestamp,profit
0,#LetsWalk_1,9.5000,15.000,2021-10-06 18:00:50,2021-12-14 03:17:48,0.578947
1,#LetsWalk_10019,1.5000,1.250,2021-08-23 22:14:34,2021-10-06 17:40:26,-0.166667
2,#LetsWalk_10020,1.9000,4.000,2021-08-13 20:09:07,2021-08-14 13:22:10,1.105263
3,#LetsWalk_10021,2.0000,2.399,2021-08-14 04:29:56,2021-08-14 07:56:12,0.199500
4,#LetsWalk_10023,5.5000,9.230,2021-08-15 17:14:04,2021-10-06 18:04:22,0.678182
...,...,...,...,...,...,...
120924,oncyber labs_1,0.8989,0.750,2021-09-20 02:05:34,2021-09-20 02:06:16,-0.165647
120925,oncyber labs_2,0.4800,0.500,2021-11-18 03:06:48,2021-11-18 03:06:48,0.041667
120926,oncyber labs_3,0.5000,0.390,2021-11-18 03:08:08,2021-11-18 03:08:08,-0.220000
120927,oncyber labs_4,0.4099,0.410,2021-12-22 19:37:07,2021-12-22 19:38:44,0.000244
